In [1]:
import pandas as pd
import numpy as np

pd.options.plotting.backend = "plotly"

In [2]:
hourly_data_path = 'students_drahi_production_consumption_hourly_completed.csv'
hourly_data = pd.read_csv(hourly_data_path)
hourly_data_index = hourly_data.set_index("datetime")

In [33]:
hourly_data_index.columns

Index(['AirTemp', 'pres', 'rain', 'rh', 'wd', 'ws', 'Global_Solar_Flux',
       'Diffuse_Solar_Flux', 'Direct_Solar_Flux', 'Downwelling_IR_Flux', 'SAA',
       'SZA', 'PAC', 'TGBT [kW]', 'kw_heater_corridor1_zone1',
       'kw_heaters_corridor_zone2', 'kw_heaters_toilets_zone2',
       'kw_heatingcoolingtotal_zone1', 'kw_heatingcoolingtotal_zone2',
       'kw_lights_zone1', 'kw_lights_zone2', 'kw_total_zone1',
       'kw_total_zone2', 'kw_ventilation_zone1', 'kw_ventilation_zone2',
       'kw_water_heater_zone2', 'plugs_zone2'],
      dtype='object')

In [3]:
from sklearn.preprocessing import FunctionTransformer

#Little hack to "freeze" transformers, since the data in the input is flattened 

def fit_and_freeze(transformer):
    fitted = [0]

    def func(x):
        if not fitted[0]:
            transformer.fit(x)
            fitted[0] = 1
        return transformer.transform(x)

    return FunctionTransformer(func)

In [42]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# AirTemp preprocessor

airtemps_pipe = Pipeline([
    ('scaler', fit_and_freeze(StandardScaler()))
])
airtemps_pipe.fit(hourly_data_index["AirTemp"].values.reshape(-1, 1))
airtemps_transformer = [(f'AirTemp-{i}', 'passthrough', [i]) for i in range(0, 4536, 27)]

#Global Solar Flux preprocessor

global_solar_flux_pipe = Pipeline([
    ('deskew', FunctionTransformer(np.vectorize(lambda x : np.log2(4+x)))),
    ('scaler', fit_and_freeze(StandardScaler()))
])
global_solar_flux_pipe.fit(hourly_data_index["Global_Solar_Flux"].values.reshape(-1, 1))
global_solar_flux_transformer = [(f'Global_Solar_Flux-{i}', 'passthrough', [i]) for i in range(6, 4536, 27)]


pres_transformers = [(f'pres-{i}', 'passthrough', [i]) for i in range(1, 4536, 27)]

In [43]:
hourly_data_index["Global_Solar_Flux"]

datetime
2022-01-01 00:00:00+00:00   -0.271
2022-01-01 01:00:00+00:00   -0.371
2022-01-01 02:00:00+00:00   -0.846
2022-01-01 03:00:00+00:00   -0.632
2022-01-01 04:00:00+00:00   -0.395
                             ...  
2023-12-31 19:00:00+00:00   -1.114
2023-12-31 20:00:00+00:00   -1.130
2023-12-31 21:00:00+00:00   -0.944
2023-12-31 22:00:00+00:00   -0.935
2023-12-31 23:00:00+00:00   -1.157
Name: Global_Solar_Flux, Length: 17520, dtype: float64

In [44]:
np.log2(4+hourly_data_index["Global_Solar_Flux"])

datetime
2022-01-01 00:00:00+00:00    1.898789
2022-01-01 01:00:00+00:00    1.859572
2022-01-01 02:00:00+00:00    1.657183
2022-01-01 03:00:00+00:00    1.751892
2022-01-01 04:00:00+00:00    1.849999
                               ...   
2023-12-31 19:00:00+00:00    1.529071
2023-12-31 20:00:00+00:00    1.521051
2023-12-31 21:00:00+00:00    1.611645
2023-12-31 22:00:00+00:00    1.615887
2023-12-31 23:00:00+00:00    1.507414
Name: Global_Solar_Flux, Length: 17520, dtype: float64

In [45]:
global_solar_flux_pipe.transform(hourly_data_index["Global_Solar_Flux"].values.reshape(-1, 1))

array([[-0.86668571],
       [-0.87926986],
       [-0.94421402],
       ...,
       [-0.95882662],
       [-0.95746525],
       [-0.99227283]])

In [5]:
time = pd.to_datetime(hourly_data['datetime']).values.astype('datetime64[s]')
power_consumption = hourly_data['kw_total_zone2'].values

dec = [] # daily energy consumption
t_dec = []

for ti, t in enumerate(time):
    tmp_t = pd.Timestamp(t)

    if np.isclose(tmp_t.hour, 0) and np.isclose(tmp_t.minute, 0):

        day_end = np.datetime64(tmp_t + pd.Timedelta(days=1))
        ind = np.where((time > tmp_t) & (time < day_end), True, False)

        if len(time[ind]) > 0 and not np.isnan(power_consumption[ind]).any():
            t_dec.append(np.datetime64(tmp_t).astype('datetime64[s]'))
            dec.append(np.trapz(power_consumption[ind], time[ind].astype(int))/3600)

t_dec = np.array(t_dec)
dec = np.array(dec)

N = 7 # N days of predictors beforehand
final_ind = []
final_hourly = []

predictor_window = pd.Timedelta(days=N)

for ti, t in enumerate(t_dec):
    tmp_t = pd.Timestamp(t)

    ind = np.where((time >= tmp_t - predictor_window) & (time < tmp_t), True, False) # finding indices within the N prior days

    bad_ind = np.isnan(hourly_data.iloc[ind, 1::].values)
    if len(time[ind]) >= 24 * N and not bad_ind.any(): # rejecting any data with NaNs; useful for the student dataset
        final_ind.append(ti)
        final_hourly.append(hourly_data.iloc[ind, 1::].values)

target_time = t_dec[final_ind]
predictors = np.array(final_hourly)

X = pd.DataFrame(predictors.reshape(len(predictors), -1))
y = pd.DataFrame(np.array(dec[final_ind]))

In [19]:
from sklearn.compose import ColumnTransformer



preprocessor = ColumnTransformer(transformers=sum([airtemps_transformers, pres_transformers], []))

preprocessor.fit_transform(X)

array([[ 10.279,  10.318,   9.901, ..., 990.737, 991.617, 992.924],
       [ 10.132,   9.669,   9.961, ..., 975.429, 977.129, 979.151],
       [ 10.627,  10.371,  10.326, ..., 988.217, 988.576, 989.088],
       ...,
       [ 10.44 ,  10.108,   9.265, ..., 995.865, 995.804, 995.709],
       [  8.507,   8.518,   8.589, ..., 994.923, 995.347, 995.869],
       [  9.449,   9.784,   9.607, ..., 986.757, 985.365, 983.953]])

In [ ]:

from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=148)

#reg = Ridge(alpha=1e8)
#reg.fit(x_train, y_train)
#y_pred = reg.predict(x_test)

In [ ]:
X

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import FunctionTransformer

# preprocessor = Pipeline(steps=[])

model = Pipeline(steps=[
    # ('preprocessor', preprocessor),
    # ('classifier', Ridge(alpha=1e8))
    ('regressor', GradientBoostingRegressor(verbose=True)),
])

model.fit(X_train, y_train.values.ravel())

In [ ]:
from skimage.metrics import mean_squared_error
from sklearn.model_selection import cross_validate

-mean_squared_error(y_test.values.ravel(), model.predict(X_test))
#np.mean(cross_validate(model, X, y.values.ravel(), cv=5, scoring='neg_mean_squared_error')['test_score'])

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {} # {'alpha': np.logspace(-2, 20, 21)}

grid = GridSearchCV(GradientBoostingRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error')

grid.fit(X, y) # using the entire dataset for validation

best_alpha = grid.best_params_['alpha']
best_rse = -grid.best_score_/np.mean((y-y.mean())**2)
print('Best alpha from validation:', best_alpha)

In [ ]:
best_model = grid.best_estimator_
np.mean(cross_validate(best_model, X, y, cv=10, scoring='neg_mean_squared_error')['test_score'])

In [ ]:
def relative_squared_error(y_pred, y_true):
    """Relative squared error (RSE; also called relative mean square error). < 1 is good, = 1 is bad, > 1 really bad."""
    return np.mean((y_pred - y_true)**2)/np.mean((y_true - y_true.mean())**2)

relative_squared_error(model.predict(X), y.values.ravel())

In [ ]:
model.predict(X).reshape(-1,1).shape

In [ ]:
wrapper = Pipeline([('wrapper', FunctionTransformer(lambda X: model.predict(X).reshape(-1,1)))])

In [ ]:
model.predict(X)

In [ ]:
model.predict(X)

In [ ]:
X.shape